## Grupo 4 - Laboratorio 1 (Python)

### Integrantes:
Katiuska Olivera Quevedo (20172533) <br/> 
Rosemery Fernandez Sanchez (20172635) <br/> 
Aliro Cabrera Florez (20152034) <br/> 
Jose Uscamayta Quispe (20195674) <br/> 

## Data analysis

In [1]:
load("C:/Users/ok235/OneDrive/Documentos/GitHub/ECO224/Labs/data/wage2015_subsample_inference.Rdata")
attach(data)

dim(data)

[1] 5150   20

In [36]:
syclg <-  scl + clg

data<-cbind(data,syclg)


In [14]:
data_Z1 <- data[data$syclg==1,]

In [19]:
library(xtable)


In [21]:
data_female <- data[data_Z1$sex==1,]
Z_female <- data_female[which(colnames(data_Z1) %in% c("lwage","sex","scl","clg","ne","mw","so","we","exp1"))]

In [22]:
data_male <- data[data_Z1$sex==0,]
Z_male <- data_male[which(colnames(data_Z1) %in% c("lwage","sex","scl","clg","ne","mw","so","we","exp1"))]

In [32]:
table <- matrix(0, 9, 3)
table[1:9,1]   <- as.numeric(lapply(data_Z,mean))
table[1:9,2]   <- as.numeric(lapply(Z_male,mean))
table[1:9,3]   <- as.numeric(lapply(Z_female,mean))
rownames(table) <- c("Log Wage","Sex","Some College","Gollage Graduate", "Northeast","Midwest","South","West","Experience")
colnames(table) <- c("All","Men","Women")
tab<- xtable(table, digits = 4)
tab

All        Men        Women     
Log Wage          2.9707867  2.9755782  2.9654732
Sex               0.4444660  0.4383309  0.4512695
Some College      0.2780583  0.2806499  0.2751843
Gollage Graduate  0.3176699  0.3124077  0.3235053
Northeast         0.2596117  0.2644018  0.2542998
Midwest           0.2965049  0.2954210  0.2977068
South             0.2161165  0.2149188  0.2174447
West              0.2277670  0.2252585  0.2305487
Experience       13.7605825 13.6124446 13.9248567

In [24]:
mean(data_female$lwage)-mean(data_male$lwage)

[1] -0.01010502

In [33]:
library(sandwich)
nocontrol.fit <- lm(lwage ~ sex)
nocontrol.est <- summary(nocontrol.fit)$coef["sex",1]
HCV.coefs <- vcovHC(nocontrol.fit, type = 'HC');
nocontrol.se <- sqrt(diag(HCV.coefs))[2] # Estimated std errors

# print unconditional effect of gender and the corresponding standard error
cat ("The estimated gender coefficient is",nocontrol.est," and the corresponding robust standard error is",nocontrol.se) 

The estimated gender coefficient is -0.03834473  and the corresponding robust standard error is 0.01590194

In [34]:

flex <- lwage ~ sex + (exp1+exp2+exp3+exp4)*(scl+clg+occ2+ind2+mw+so+we)

#   Note that ()*() operation in formula objects in R creates a formula of the sort:
#  (exp1+exp2+exp3+exp4)+ (shs+hsg+scl+clg+occ2+ind2+mw+so+we) +  (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)
#  This is not intuitive at all, but that's what it does.

control.fit <- lm(flex, data=data_Z1)
control.est <- summary(control.fit)$coef[2,1]

summary(control.fit)

cat("Coefficient for OLS with controls", control.est)

HCV.coefs <- vcovHC(control.fit, type = 'HC');
control.se <- sqrt(diag(HCV.coefs))[2] # Estimated std errors


Call:
lm(formula = flex, data = data_Z1)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.87897 -0.27894 -0.00777  0.25823  2.85755 

Coefficients: (6 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.6856233  0.5060297   7.283  4.2e-13 ***
sex         -0.0530623  0.0193532  -2.742 0.006149 ** 
exp1        -0.1840774  0.1961271  -0.939 0.348036    
exp2         4.0888929  2.3488332   1.741 0.081824 .  
exp3        -2.2064389  1.0068217  -2.191 0.028497 *  
exp4         0.3378726  0.1370444   2.465 0.013744 *  
scl         -0.2498673  0.1237850  -2.019 0.043627 *  
clg                 NA         NA      NA       NA    
occ22        0.2151993  0.1582208   1.360 0.173900    
occ23        0.0487642  0.2095297   0.233 0.815986    
occ24        0.0281449  0.2300084   0.122 0.902619    
occ25       -0.2711807  0.3944166  -0.688 0.491793    
occ26       -0.2000530  0.2705614  -0.739 0.459725    
occ27       -0.1203371  0.41

Coefficient for OLS with controls -0.05306234

In [35]:
# Partialling-Out using ols

# models
flex.y <- lwage ~  (exp1+exp2+exp3+exp4)*(scl+clg+occ2+ind2+mw+so+we) # model for Y
flex.d <- sex ~ (exp1+exp2+exp3+exp4)*(scl+clg+occ2+ind2+mw+so+we) # model for D

# partialling-out the linear effect of W from Y
t.Y <- lm(flex.y, data=data_Z1)$res
# partialling-out the linear effect of W from D
t.D <- lm(flex.d, data=data_Z1)$res

# regression of Y on D after partialling-out the effect of W
partial.fit <- lm(t.Y~t.D)
partial.est <- summary(partial.fit)$coef[2,1]

cat("Coefficient for D via partialling-out", partial.est)

# standard error
HCV.coefs <- vcovHC(partial.fit, type = 'HC')
partial.se <- sqrt(diag(HCV.coefs))[2]

# confidence interval
confint(partial.fit)[2,]

Coefficient for D via partialling-out -0.05306234

2.5 %      97.5 % 
-0.08957070 -0.01655398